In [26]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import collections
import random
import matplotlib.pyplot as plt
import pyautogui as pag
import PIL
import torchvision.transforms as transforms
import cv2
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import time
import seaborn as sns
from torch.distributions import Categorical

pag.FAILSAFE = False


import argparse
import torch

import torch.multiprocessing as _mp
from torch.distributions import Categorical
import torch.nn.functional as F
import numpy as np
import shutil
from torch.utils.tensorboard import SummaryWriter


from random import sample

In [27]:
#env.py
class ENV():
    def __init__(self):
        # screenshot의 위치 지정, 클래스 생성할때 가져오기

        # winflg와 lose flag 존재해야 함. 0으로 하는 건 grayscale
        self.winFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/weWin.png', 0)
        self.loseFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemyWin.png', 0)
        # nocard flag
        self.nocardFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/test1/nocard.png')
        # noElixir flag
        self.noelixirFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/test1/noElixir.png')
        self.startGameFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/battleStart.png')
        self.finishFlag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/battleFinish.png')
        self.enemy1Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy1.png')
        self.enemy2Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy2.png')
        self.enemy3Flag = cv2.imread('/home/jnu/Desktop/Reinforce/Royale/screenshot/cr_test/enemy3.png')
    def return_state(self, img):
        # 스크린 샷을 인자로 받아와서 모델에 넣을 수 있도록 tensor로 변환
        tf = transforms.ToTensor()
        img_t = tf(img) # time.sleep(0.3)
        img_t = img_t.unsqueeze(0)
        # img_t = img_t.permute(1, 0, 2, 3)

        return img_t

    def check_finish(self,img):
        finishMessage = cv2.cvtColor(self.finishFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(finishMessage, img, cv2.TM_CCOEFF_NORMED)
        if (np.max(ratio) > 0.90):
            return 1

        else:
            return 0

    def check_win(self, img):
        # 게임이 이겼는지 확인, screenshot을 가져와서 우리가 원하는 크기로 잘라서 확인
        # img = np.array(img)
        checkFlag1 = np.array(img.crop((225, 335, 280, 365)))
        checkFlag1 = cv2.cvtColor(checkFlag1, cv2.COLOR_BGR2GRAY)
        win_check = cv2.matchTemplate(checkFlag1, self.winFlag, cv2.TM_CCOEFF_NORMED)
        if win_check > 0.8:
            return 1

        else:
            return 0
           
    def check_lose(self, img):
        # 게임이 졌는지 확인, screenshot을 가져와서 우리가 원하는 크기로 잘라서 확인
        # img = np.array(img)
        checkFlag2 = np.array(img.crop((225, 85, 280, 115)))
        checkFlag2 = cv2.cvtColor(checkFlag2, cv2.COLOR_BGR2GRAY)
        lose_check = cv2.matchTemplate(checkFlag2, self.loseFlag, cv2.TM_CCOEFF_NORMED)
        if lose_check > 0.8:
            return 1

        else:
            return 0

    def check_card(self, img):
        # 카드를 선택하지 않았는지 확인, screenshot을 가져와서 init에 지정된 nocard 이미지와 비교하여 reward 부여
        nocard = cv2.cvtColor(self.nocardFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(nocard, img, cv2.TM_CCOEFF_NORMED)

        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def check_elixir(self, img):
        noElixir = cv2.cvtColor(self.noelixirFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(noElixir, img, cv2.TM_CCOEFF_NORMED)

        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def checkET1(self, img):
        score1 = 0
        checkFlag1 = np.array(img.crop((105, 138, 155, 139)))

        for i in range(50):
            if (checkFlag1[0][i][0] >= 96):
                score1 += 1

        score1 = score1 * 2
        return score1

    def checkET2(self, img):
        score2 = 0
        checkFlag2 = np.array(img.crop((371, 138, 421, 139)))

        for i in range(50):
            if (checkFlag2[0][i][0] >= 96):
                score2 += 1

        score2 = score2 * 2
        return score2

    def checkGameStart(self, img):
        startMessage = cv2.cvtColor(self.startGameFlag, cv2.COLOR_BGR2GRAY)
        img = np.array(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(startMessage, img, cv2.TM_CCOEFF_NORMED)
        if (np.max(ratio) > 0.90):
            # print(np.max(ratio))
            return 1

        else:
            return 0

    def enemy1(self, img):
        crownFlag1 = cv2.cvtColor(self.enemy1Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag1, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0
        
    def enemy2(self, img):
        crownFlag2 = cv2.cvtColor(self.enemy2Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag2, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0
        
    def enemy3(self, img):
        crownFlag3 = cv2.cvtColor(self.enemy3Flag, cv2.COLOR_BGR2GRAY)
        img1 = np.array(img)
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        ratio = cv2.matchTemplate(crownFlag3, img1, cv2.TM_CCOEFF_NORMED)
        #print(np.max(ratio))
        if (np.max(ratio) > 0.90):
            return 1
        else:
            return 0

    # def checkOT1
    # def checkOT1
    # def checkOT1

    # 우리 타워와 상대 타워의 hp를 확인하여 reward 부여

    def retryGame(self):
        time.sleep(3)
        pag.click((2860, 875))
        time.sleep(5)
        pag.click((3070, 185))
        time.sleep(5)
        pag.click((2920, 385))
        time.sleep(3)
        pag.click((2950, 615))



In [28]:
#model.py
class PPO(nn.Module):
    def __init__(self, num_actions):
        super(PPO, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 16, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 3, stride=2, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(inplace=True))
        self.actor_linear = nn.Sequential(nn.Linear(462848, 256),
                                          nn.ReLU(inplace=True),
                                          nn.Linear(256, num_actions))
        self.critic_linear = nn.Sequential(nn.Linear(462848, 256),
                                          nn.ReLU(inplace=True),
                                          nn.Linear(256, 1))

        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = x.reshape(x.size(0), -1)
        #x = F.softmax(x, dim=softmax_dim)
        return self.actor_linear(x), self.critic_linear(x)

In [29]:
#train.py envs = MultipleEnvironments(opt.world, opt.stage, opt.action_type, opt.num_processes)
env = ENV()

In [30]:
#train.py model = PPO(envs.num_states, envs.num_actions)
model = PPO(11)

In [31]:
action_list = [[2720, 550], [2860, 550], [3000, 550], [2720 ,660], [2860, 660], [3000, 660], [2765, 900], [2855, 930], [2950, 900], [3055, 900], [-1, -1]]
action_list_name = {0:'left top', 1:'center top', 2:'right top', 3:'right bottom', 4:'center bottom', 5:'center right', 6:'card 1', 7:'card 2', 8:'card 3', 9:'card 4', 10:'rest action'}


In [32]:
#optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr)

optimizer = optim.Adam(model.parameters(), lr = 0.0005)

In [33]:
writer = SummaryWriter()

In [34]:
#train.py curr_episode = 0
curr_episode = 0
train_time = 0

In [37]:
while True:
    curr_episode += 1
    old_log_policies = []
    actions = []
    values = []
    states = []
    rewards = []
    dones = []
    
    #reward 초기화
    reward = 0
    
    #done 초기화
    done = 0
    
    # enemy tower score / score1 = left, score2 = right
    score1 = 100
    score2 = 100
    
    #img to curr states
    img = pag.screenshot(region=(2605, 100, 510, 900))
    curr_states = env.return_state(img)
    
    num_local_steps = 0
    
    #set start_flag
    start_flag = 0
    
    while(1):
    #for _ in range(40):
        #opt 기본 num_local_steps = 512
        
        #게임 시작 확인 전까지 멈춤 지시
        while (start_flag == 0):
            # 스크린샷 찍기
            img = pag.screenshot(region=(2605, 100, 510, 900))

            if (env.checkGameStart(img)):
                # print("game start")
                start_flag = 1
                time.sleep(3)
                break
        
        
        #num_local_step_count
        num_local_steps += 1
        
        #현재 이미지 캡처 후 변환
        img = pag.screenshot(region=(2605, 100, 510, 900))
        curr_states = env.return_state(img)
        
        # 승리 확인
        if (env.check_win(img)):
            print("win")
            reward += 5000
            done = 1
        # 패배 확인
        if (env.check_lose(img)):
            print("lose")
            reward -= 5000
            done = 1
        # no card확인
        if (env.check_card(img)):
            reward -= 1
            print("no card")

        # no elixir확인
        if (env.check_elixir(img)):
            reward -= 1
            print("no Elixir")
            
        if (env.enemy1(img)):
            reward -= 100
            print("crown - 1")
        
        if (env.enemy2(img)):
            reward -= 100
            print("crown - 2")
        
        if (env.enemy3(img)):
            reward -= 5000
            done = 1
            print("crown - 3")


        # enemy tower reward calculate
        score1_now = env.checkET1(img)
        score2_now = env.checkET2(img)
        #print(f"score1 = {score1}, score1_now = {score1_now}")
        #print(f"score2 = {score2}, score2_now = {score2_now}")
        print(f"left tower HP = {score1}, right tower HP = {score2}")
        if (score1_now < score1):
            reward += 5 * (score1 - score1_now)
            score1 = score1_now

        if (score2_now < score2):
            reward += 5 * (score2 - score2_now)
            score2 = score2_now
        
        
        
        #train.py states.append(curr_states)
        states.append(curr_states)
        
        #logits, value = model(curr_states)
        logits, value = model(curr_states)
        
        #values.append(value.squeeze())
        values.append(value.squeeze())
        
        #policy = F.softmax(logits, dim=1)
        policy = F.softmax(logits, dim=1)
        
        #old_m = Categorical(policy)
        old_m = Categorical(policy)
        
        #action = old_m.sample()
        action = old_m.sample()
        print(action)
        
        #pag 클릭. rest action은 -1 -1 이므로 실행하지 않음
        if(action!=10):
            # 화면 클릭
            pag.click(action_list[action][0], action_list[action][1])
        
        #actions.append(action)
        actions.append(action)
        
        #old_log_policy = old_m.log_prob(action)
        old_log_policy = old_m.log_prob(action)
        
        #old_log_policies.append(old_log_policy)
        old_log_policies.append(old_log_policy)
        
        #단위시간당 reward 감소
        reward -= 1
        
        
        
        #state 불러오기
        img = pag.screenshot(region=(2605, 100, 510, 900))
        state = env.return_state(img)
        
       
        
        curr_states = curr_states.numpy()
        #print(curr_states)
        #train.py state = torch.from_numpy(np.concatenate(state, 0))
        test1 = np.concatenate(curr_states, 0)
        state = torch.from_numpy(np.concatenate(curr_states, 0))
        
        #train.py reward.append, dones.append
        rewards.append(reward)
        dones.append(done)
        
        #print action and reward
        print('action =', action_list_name[a], '    index = ', action_list[a][0], action_list[a][1])
        print("reward = ", reward)
        
        #curr_states = state
        img = pag.screenshot(region=(2605, 100, 510, 900))
        curr_states = env.return_state(img)
        
        if(done == 1):
            break
    
    #print("after the corner1")
    
    _, next_value, = model(curr_states)
    next_value = next_value.squeeze()
    old_log_policies = torch.cat(old_log_policies).detach()
    actions = torch.cat(actions)
    #print(f"values = {values}")
    #values = torch.cat(values).detach()
    values = torch.tensor(values).detach()
    states = torch.cat(states)
    gae = 0
    R = []
    #print("after the corner2")
    
    for value, reward, done in list(zip(values, rewards, dones))[::-1]:
        #opt.gamma 0.9, opt.tau = 1.0
        gamma = 0.9
        tau = 1.0
        gae = gae * gamma * tau
        gae = gae + reward + gamma * next_value.detach() * (1 - done) - value.detach()
        next_value = value
        R.append(gae + value)
    R = R[::-1]
    #R = torch.cat(R).detach()
    R = torch.tensor(R).detach()
    advantages = R - values
    
    #print("after the corner3")
    #opt.num_epochs = 10
    num_epochs = 3
    for i in range(num_epochs):
        #opt.num_local_steps = 512 but 우리는 몇 step을 했는지를 카운트하여 대체
        #opt.num_processes은 8
        num_processes = 8
        
        #opt.batch_size = 16
        batch_size = 2
        
        #indice = torch.randperm(num_local_steps * num_processes)
        indice = torch.randperm(num_local_steps)
        
        
        for j in range(batch_size):
            #이것은 어떻게 가능한가?
#             batch_indices = indice[
#                             int(j * (num_local_steps * num_processes / batch_size)): int((j + 1) * (
#                                     num_local_steps * num_processes / batch_size))]

            #indice copy concept
            #states 기준으로 편집
            indice2 = sample(range(num_local_steps), int(num_local_steps/batch_size))
            #이렇게 짜면 후반 부담이 늘 수 있음. 개선 방안 필요.
        
            batch_indices = torch.tensor(indice2)
    
#             print(f"start page = {int(j * (num_local_steps * num_processes / batch_size))},\
#                             end page = {int((j + 1) * (num_local_steps * num_processes / batch_size))}")
#             print(f"0 size = {indice.size()}")
#             print(f"1 size = {batch_indices}")
#             print(f"2 size = {len(states)}")
#             print(f"3 size = {states.shape}")
#             print(f"4 size = {states[batch_indices]}")
            
            logits, value = model(states[batch_indices])
            
            new_policy = F.softmax(logits, dim=1)
            new_m = Categorical(new_policy)
            new_log_policy = new_m.log_prob(actions[batch_indices])
            ratio = torch.exp(new_log_policy - old_log_policies[batch_indices])
            
            #opt.epsilon = 0.2
            epsilon = 0.2
            
            actor_loss = -torch.mean(torch.min(ratio * advantages[batch_indices],
                                               torch.clamp(ratio, 1.0 - epsilon, 1.0 + epsilon) *
                                               advantages[
                                                   batch_indices]))
            # critic_loss = torch.mean((R[batch_indices] - value) ** 2) / 2
            critic_loss = F.smooth_l1_loss(R[batch_indices], value.squeeze())
            entropy_loss = torch.mean(new_m.entropy())
            
            #opt.beta = 0.01
            beta = 0.01
            
            total_loss = actor_loss + critic_loss - beta * entropy_loss
            writer.add_scalar('critic_loss/train', critic_loss, curr_episode)
            writer.add_scalar('entropy_loss/train', entropy_loss, curr_episode)
            writer.add_scalar('total_loss/train', total_loss, curr_episode)
            optimizer.zero_grad()
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            train_time += 1
            print(f"how many time to train? = {train_time}")
            #opt.save_interval = 50
            save_interval = 50
        print("Episode: {}. Total loss: {}, save_interval = {}".format(curr_episode, total_loss, curr_episode % save_interval))
        env.retryGame()

left tower HP = 100, right tower HP = 100
tensor([9])
left tower HP = 100, right tower HP = 100
tensor([5])
left tower HP = 100, right tower HP = 100
tensor([4])
no card
left tower HP = 100, right tower HP = 100
tensor([7])
no card
left tower HP = 100, right tower HP = 100
tensor([0])
left tower HP = 100, right tower HP = 100
tensor([1])
no card
left tower HP = 100, right tower HP = 100
tensor([7])
no card
left tower HP = 100, right tower HP = 100
tensor([6])
left tower HP = 100, right tower HP = 100
tensor([1])
left tower HP = 100, right tower HP = 100
tensor([7])
left tower HP = 100, right tower HP = 100
tensor([10])
left tower HP = 100, right tower HP = 100
tensor([3])
left tower HP = 100, right tower HP = 100
tensor([10])
left tower HP = 100, right tower HP = 100
tensor([4])
left tower HP = 100, right tower HP = 100
tensor([0])
left tower HP = 100, right tower HP = 100
tensor([7])
left tower HP = 100, right tower HP = 100
tensor([10])
left tower HP = 100, right tower HP = 100
tenso

KeyboardInterrupt: 

In [10]:
a = torch.tensor([0])

In [13]:
print(action_list[a])

[2720, 550]
